In [1]:
import torch
from torch import nn

In [3]:
net = nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,1))
X=torch.rand(2,4)
net(X)

tensor([[-0.0075],
        [ 0.0283]], grad_fn=<AddmmBackward0>)

In [4]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.3001, -0.0884, -0.0782, -0.3337, -0.1463, -0.0076, -0.1172,  0.2769]])), ('bias', tensor([0.3090]))])


In [5]:
print(type(net[2].bias))

<class 'torch.nn.parameter.Parameter'>


In [6]:
print(net[2].bias)

Parameter containing:
tensor([0.3090], requires_grad=True)


In [8]:
print(net[2].bias.data)
print(net[2].bias.grad)

tensor([0.3090])
None


In [12]:
print(*[(name,param.shape) for name,param in net.named_parameters()])

('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [13]:
net.state_dict()

OrderedDict([('0.weight',
              tensor([[ 0.1826, -0.0592, -0.4062,  0.0278],
                      [-0.3792, -0.2628, -0.1767, -0.0453],
                      [ 0.4230,  0.3422, -0.4257, -0.1628],
                      [-0.0454,  0.1027,  0.1218,  0.0538],
                      [-0.4287, -0.3658, -0.0613, -0.1513],
                      [ 0.0109, -0.4511, -0.3009, -0.1766],
                      [ 0.4074,  0.3734, -0.3476,  0.4378],
                      [ 0.3213, -0.0462,  0.2348, -0.1275]])),
             ('0.bias',
              tensor([ 0.1202,  0.4573, -0.0059,  0.4396,  0.1105,  0.0623,  0.4076, -0.2640])),
             ('2.weight',
              tensor([[ 0.3001, -0.0884, -0.0782, -0.3337, -0.1463, -0.0076, -0.1172,  0.2769]])),
             ('2.bias', tensor([0.3090]))])

In [14]:
net.state_dict()['2.bias'].data

tensor([0.3090])

In [15]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                        nn.Linear(8, 4), nn.ReLU())

def block2():
    net =nn.Sequential()
    for i in range(4):
        net.add_module(f'block{i}',block1())
    return net

In [16]:
rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[0.4855],
        [0.4855]], grad_fn=<AddmmBackward0>)

In [17]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


# Built-in Initialization

In [18]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight,mean=0,std=0.01)
        nn.init.zeros_(m.bias)

In [19]:
net.apply(init_normal)
net.state_dict()

OrderedDict([('0.weight',
              tensor([[ 0.0025,  0.0055, -0.0103, -0.0014],
                      [-0.0091, -0.0085, -0.0062,  0.0010],
                      [ 0.0076,  0.0006, -0.0058, -0.0120],
                      [ 0.0038, -0.0007, -0.0049,  0.0090],
                      [ 0.0054,  0.0025,  0.0002,  0.0059],
                      [-0.0164, -0.0216,  0.0055, -0.0092],
                      [ 0.0014, -0.0126,  0.0005, -0.0093],
                      [-0.0101, -0.0020, -0.0006,  0.0068]])),
             ('0.bias', tensor([0., 0., 0., 0., 0., 0., 0., 0.])),
             ('2.weight',
              tensor([[ 0.0051, -0.0209,  0.0031,  0.0047,  0.0070, -0.0103, -0.0156, -0.0256]])),
             ('2.bias', tensor([0.]))])

In [20]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [21]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.4391,  0.4716, -0.6771, -0.2676])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [24]:
net[0].weight.data[:]=42
net[0].weight.data

tensor([[42., 42., 42., 42.],
        [42., 42., 42., 42.],
        [42., 42., 42., 42.],
        [42., 42., 42., 42.],
        [42., 42., 42., 42.],
        [42., 42., 42., 42.],
        [42., 42., 42., 42.],
        [42., 42., 42., 42.]])

# Tied Parameters

In [25]:
shared=nn.Linear(8,8)
net =nn.Sequential(nn.Linear(4,8),nn.ReLU(),
                                shared,nn.ReLU(),
                                shared,nn.ReLU(),
                                nn.Linear(8,1))
net(X)

tensor([[-0.2075],
        [-0.2058]], grad_fn=<AddmmBackward0>)

In [28]:
print(net[2].weight.data[0]==net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])


In [29]:
net[2].weight.data[0,0]=100
print(net[4].weight.data[0,0]==100)

tensor(True)
